# 2D slices through the simulation box

In [ ]:
%pylab widget
#%pylab ipympl
#%pylab inline
from plotting_settings import *

import os
cwd = os.getcwd()
plot_dir = os.path.join(cwd, 'plots')
if not os.path.exists(plot_dir):
    os.mkdir(plot_dir)
    
from ppmpy import ppm

import logging
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

data_path = '/data/CoCo/'

In [ ]:
vort_ref = 1.8
vort_ref_str = r'{:.2f}'.format(vort_ref)
X1_ref = 0.14
X1_ref_str = r'{:.3f}'.format(X1_ref)
views = (
         {'q':'X1', 'sl':'Z-0.000', 'qlim':(0.5, 1.), 'cmap':'PuOr', 
          'lbl':r'$\tanh\left[ X_1(z = 0) \,/\, '+X1_ref_str+r' \right]$'},
         {'q':'A', 'sl':'Z-0.000', 'qlim':(-2.900, 2.999), 'cmap':'bwr',
          'lbl':r'$\left[ \left(A - \overline{A} \right) \,/\, A_\mathrm{rms} \right]_{z=0}$'},
         {'q':'VORT', 'sl':'Z-0.000', 'qlim':(0., 1.), 'cmap':'viridis',
          'lbl':r'$\tanh\left[ |\omega(z = 0)| \,/\, '+vort_ref_str+r' \right]$'},
         {'q':'VY', 'sl':'Y-1.700', 'qlim':(-0.065, 0.065), 'cmap':'BrBG', 
          'lbl':r'$v_y(y = 1.7)$'},
        )
dmp = 200
codes = ('FLASH', 'MUSIC', 'PPMSTAR', 'PROMPI', 'SLH')
res = 256

fig = figure(figsize=(0.95*pwidth, 1.22*pwidth), dpi=scrdpi)
gs = mpl.gridspec.GridSpec(6, 4, height_ratios=[1, 1, 1, 1, 1, 0.1])
for ic, code in enumerate(codes):
    for iv in range(len(views)):
        rid = code+'-'+str(res)
        fn = '{:s}-{:d}-{:04d}-{:s}-{:s}.npy'.format(code, res, dmp,
             views[iv]['q'], views[iv]['sl'])
        slice_path = os.path.join(data_path, '2D-slices', code, rid,
                     '{:04d}'.format(dmp), fn)
        data = np.load(slice_path)
        
        run_path = os.path.join(data_path, '1D-profiles', code, rid)
        rps = ppm.RprofSet(run_path, geometry='cartesian', verbose=2)
        rp = rps.get_dump(dmp)
        
        if views[iv]['sl'][0] in ('Y', 'Z'):
            data = np.transpose(data)[::-1,:]
        else:
            data = data[::-1,:]
        
        ax = subplot(gs[ic, iv])
        qmin = views[iv]['qlim'][0]
        qmax = views[iv]['qlim'][1]
        
        if views[iv]['q'] == 'VORT':
            data = np.tanh(data/vort_ref)
        if views[iv]['q'] == 'A':
            #adata = np.mean(data, axis=1)
            #adata = np.transpose(np.reshape(np.tile(adata, res), (res, res)))
            #rmsdata = np.mean((data - adata)**2, axis=1)**0.5
            #rmsdata = np.transpose(np.reshape(np.tile(rmsdata, res), (res, res)))
            #data = (data - adata)/rmsdata
            adata = rp.get('A')
            adata = np.transpose(np.reshape(np.tile(adata, res), (res, res)))
            rmsdata = rp.get('STDEV_A')
            rmsdata = np.transpose(np.reshape(np.tile(rmsdata, res), (res, res)))
            data = (data - adata)/rmsdata
        if views[iv]['q'] == 'X1':
            data = np.tanh(data/X1_ref)
        
        norm = mpl.colors.Normalize(vmin=qmin, vmax=qmax)
        cmap = views[iv]['cmap']
        pos = ax.imshow(data, cmap=cmap, norm=norm)
            
        ttl = r'{:s}, ${:d}^3$'.format(code, res)
        if iv == 0:
            ax.set_ylabel(ttl, fontsize=textsize)
        ax.xaxis.set_ticks([])
        ax.yaxis.set_ticks([])
        
        if ic == len(codes) - 1:
            ax0 = subplot(gs[-1, iv])
            fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), \
                         cax=ax0, orientation='horizontal', \
                         label=views[iv]['lbl'])

    subplots_adjust(left=0.035, right=0.985, bottom=0.045, top=0.995, hspace=0.05, wspace=0.05)
    savefig(os.path.join(plot_dir, 'slices_{:d}_{:04d}.pdf'.format(res, dmp)), dpi=savedpi)